## Gold Layer - ML Training, Risk Scoring & Business Decisions
Purpose: Train explainable ML model and generate loan approval decisions


###Gold Phase 1 - Model Training & Evaluation
Purpose: Train explainable ML model and validate performance


In [0]:
# Load Silver layer features for ML training

df_silver = spark.table("silver_loan_features")

df_silver.printSchema()
df_silver.show(5)


root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- total_income: double (nullable = true)
 |-- loan_to_income_ratio: double (nullable = true)
 |-- credit_flag: integer (nullable = true)
 |-- employment_flag: integer (nullable = true)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+------------+--------------------+-----------+---------------+
| Loan_ID

In [0]:
# Create binary target variable for loan approval (1 = approved, 0 = rejected)

from pyspark.sql.functions import when

df_labeled = df_silver.withColumn(
    "loan_approved_flag",
    when(df_silver.Loan_Status == "Y", 1).otherwise(0)
)

df_labeled.select("Loan_Status", "loan_approved_flag").show(5)


+-----------+------------------+
|Loan_Status|loan_approved_flag|
+-----------+------------------+
|          N|                 0|
|          Y|                 1|
|          Y|                 1|
|          Y|                 1|
|          Y|                 1|
+-----------+------------------+
only showing top 5 rows


In [0]:
# Create a simple heuristic risk band using loan-to-income ratio
# Used only for baseline understanding, not final decisions

from pyspark.sql.functions import col

df_risk_base = df_labeled.withColumn(
    "initial_risk_band",
    when(col("loan_to_income_ratio") <= 0.25, "LOW")
    .when(col("loan_to_income_ratio") <= 0.50, "MEDIUM")
    .otherwise("HIGH")
)

df_risk_base.select(
    "loan_to_income_ratio",
    "initial_risk_band"
).show(10)


+--------------------+-----------------+
|loan_to_income_ratio|initial_risk_band|
+--------------------+-----------------+
|  0.0210146117222131|              LOW|
|               0.022|              LOW|
|0.024286581663630843|              LOW|
|              0.0235|              LOW|
|0.027774888172266723|              LOW|
|0.024681735515718367|              LOW|
| 0.02851985559566787|              LOW|
| 0.03036876355748373|              LOW|
|0.014658322483094628|              LOW|
|0.017948717948717947|              LOW|
+--------------------+-----------------+
only showing top 10 rows


In [0]:
# Select numerical and engineered features for ML model training

ml_features = [
    "total_income",
    "loan_to_income_ratio",
    "credit_flag",
    "employment_flag",
    "LoanAmount",
    "Loan_Amount_Term"
]

df_ml = df_risk_base.select(
    *ml_features,
    "loan_approved_flag"
)

df_ml.show(5)


+------------+--------------------+-----------+---------------+----------+----------------+------------------+
|total_income|loan_to_income_ratio|credit_flag|employment_flag|LoanAmount|Loan_Amount_Term|loan_approved_flag|
+------------+--------------------+-----------+---------------+----------+----------------+------------------+
|      6091.0|  0.0210146117222131|          1|              0|       128|             360|                 0|
|      3000.0|               0.022|          1|              1|        66|             360|                 1|
|      4941.0|0.024286581663630843|          1|              0|       120|             360|                 1|
|      6000.0|              0.0235|          1|              0|       141|             360|                 1|
|      9613.0|0.027774888172266723|          1|              1|       267|             360|                 1|
+------------+--------------------+-----------+---------------+----------+----------------+------------------+
o

In [0]:
# Define ML feature columns (used consistently throughout notebook)

feature_cols = [
    "total_income",
    "loan_to_income_ratio",
    "credit_flag",
    "employment_flag",
    "LoanAmount",
    "Loan_Amount_Term"
]


In [0]:
# Check for null values in ML features

from pyspark.sql.functions import sum, col

df_ml.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in feature_cols
]).show()


+------------+--------------------+-----------+---------------+----------+----------------+
|total_income|loan_to_income_ratio|credit_flag|employment_flag|LoanAmount|Loan_Amount_Term|
+------------+--------------------+-----------+---------------+----------+----------------+
|           0|                   0|          0|              0|         0|              14|
+------------+--------------------+-----------+---------------+----------+----------------+



In [0]:
# Remove rows with missing feature values

df_ml_non_null = df_ml.dropna(subset=feature_cols)

print("Rows before:", df_ml.count())
print("Rows after:", df_ml_non_null.count())


Rows before: 543
Rows after: 529


In [0]:
# Assemble ML feature vector

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_ml_ready = assembler.transform(df_ml_non_null).select(
    "features",
    "loan_approved_flag"
)

df_ml_ready.show(5)


+--------------------+------------------+
|            features|loan_approved_flag|
+--------------------+------------------+
|[6091.0,0.0210146...|                 0|
|[3000.0,0.022,1.0...|                 1|
|[4941.0,0.0242865...|                 1|
|[6000.0,0.0235,1....|                 1|
|[9613.0,0.0277748...|                 1|
+--------------------+------------------+
only showing top 5 rows


In [0]:
# Split dataset into training and test sets

train_df, test_df = df_ml_ready.randomSplit([0.8, 0.2], seed=42)

print("Train size:", train_df.count())
print("Test size:", test_df.count())


Train size: 447
Test size: 82


In [0]:
# Train explainable Logistic Regression model

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="loan_approved_flag"
)

lr_model = lr.fit(train_df)


In [0]:
# Generate predictions on test data

predictions = lr_model.transform(test_df)

display(
    predictions.select(
        "loan_approved_flag",
        "probability",
        "prediction"
    )
)


loan_approved_flag,probability,prediction
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.20508596255387151"",""0.7949140374461285""]}",1.0
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.21876664896309472"",""0.7812333510369053""]}",1.0
0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9250547214490589"",""0.07494527855094113""]}",0.0
0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.223552711174114"",""0.776447288825886""]}",1.0
0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.4071748073092742"",""0.5928251926907258""]}",1.0
0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.18106850279614214"",""0.8189314972038578""]}",1.0
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.23805108768693403"",""0.7619489123130659""]}",1.0
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.13918457039929405"",""0.860815429600706""]}",1.0
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.25205556918668953"",""0.7479444308133105""]}",1.0
1,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.22599396039425018"",""0.7740060396057498""]}",1.0


In [0]:
# Extract approval probability from prediction vector

from pyspark.ml.functions import vector_to_array

predictions_with_prob = predictions.withColumn(
    "approval_probability",
    vector_to_array(col("probability"))[1]
)

display(
    predictions_with_prob.select(
        "loan_approved_flag",
        "approval_probability",
        "prediction"
    )
)


loan_approved_flag,approval_probability,prediction
1,0.7949140374461285,1.0
1,0.7812333510369053,1.0
0,0.07494527855094113,0.0
0,0.776447288825886,1.0
0,0.5928251926907258,1.0
0,0.8189314972038578,1.0
1,0.7619489123130659,1.0
1,0.860815429600706,1.0
1,0.7479444308133105,1.0
1,0.7740060396057498,1.0


In [0]:
# Evaluate model using AUC

from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="loan_approved_flag",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(predictions_with_prob)
print("Test AUC:", auc)


Test AUC: 0.733477011494253


In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.mlflow_volume;


In [0]:
import os

os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/mlflow_volume/mlflow_tmp"


In [0]:
# Log model and metrics using MLflow (Unity Catalog compatible)

import mlflow
import mlflow.spark
import os

# Required for UC-enabled Databricks clusters
os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/default/mlflow_volume/mlflow_tmp"

with mlflow.start_run(run_name="Loan_Approval_Logistic_Regression"):
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("features", ",".join(feature_cols))
    mlflow.log_metric("AUC", auc)
    mlflow.spark.log_model(
        lr_model,
        artifact_path="loan_approval_model"
    )


2026/01/30 19:06:58 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/30 19:07:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-17d798b7-d81c-4db8-beb7-d2/tmp3y2x75hu/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/30 19:07:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [0]:
# Extract feature coefficients for explainability

coefficients = lr_model.coefficients.toArray()

from pyspark.sql import Row

importance_df = spark.createDataFrame(
    [Row(feature=f, coefficient=float(c)) for f, c in zip(feature_cols, coefficients)]
)

display(importance_df)


feature,coefficient
total_income,-3.4271034034609086E-5
loan_to_income_ratio,-31.29693280952568
credit_flag,3.7116637229982343
employment_flag,-0.20494867845434883
LoanAmount,0.0011555768114786171
Loan_Amount_Term,-0.0010166313231458966


In [0]:
#Convert Probabilities to Risk Bands (Test Data)

from pyspark.sql.functions import when, col

df_decisions = predictions_with_prob.withColumn(
    "risk_band",
    when(col("approval_probability") >= 0.75, "LOW_RISK")
    .when(col("approval_probability") >= 0.50, "MEDIUM_RISK")
    .otherwise("HIGH_RISK")
)

display(df_decisions.select("approval_probability", "risk_band"))


approval_probability,risk_band
0.7949140374461285,LOW_RISK
0.7812333510369053,LOW_RISK
0.07494527855094113,HIGH_RISK
0.776447288825886,LOW_RISK
0.5928251926907258,MEDIUM_RISK
0.8189314972038578,LOW_RISK
0.7619489123130659,LOW_RISK
0.860815429600706,LOW_RISK
0.7479444308133105,MEDIUM_RISK
0.7740060396057498,LOW_RISK


In [0]:
# Convert risk bands into business decisions

df_decisions = df_decisions.withColumn(
    "decision",
    when(col("risk_band") == "LOW_RISK", "AUTO_APPROVE")
    .when(col("risk_band") == "MEDIUM_RISK", "MANUAL_REVIEW")
    .otherwise("REJECT")
)

display(df_decisions.select("approval_probability", "risk_band", "decision"))



approval_probability,risk_band,decision
0.7949140374461285,LOW_RISK,AUTO_APPROVE
0.7812333510369053,LOW_RISK,AUTO_APPROVE
0.07494527855094113,HIGH_RISK,REJECT
0.776447288825886,LOW_RISK,AUTO_APPROVE
0.5928251926907258,MEDIUM_RISK,MANUAL_REVIEW
0.8189314972038578,LOW_RISK,AUTO_APPROVE
0.7619489123130659,LOW_RISK,AUTO_APPROVE
0.860815429600706,LOW_RISK,AUTO_APPROVE
0.7479444308133105,MEDIUM_RISK,MANUAL_REVIEW
0.7740060396057498,LOW_RISK,AUTO_APPROVE


In [0]:
# Persist baseline Gold decisions (test data)

df_gold_baseline = df_decisions.select(
    "approval_probability",
    "risk_band",
    "decision",
    "loan_approved_flag"
)

df_gold_baseline.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_loan_decisions")

display(spark.sql("SELECT * FROM gold_loan_decisions"))



approval_probability,risk_band,decision,loan_approved_flag
0.7949140374461285,LOW_RISK,AUTO_APPROVE,1
0.7812333510369053,LOW_RISK,AUTO_APPROVE,1
0.07494527855094113,HIGH_RISK,REJECT,0
0.776447288825886,LOW_RISK,AUTO_APPROVE,0
0.5928251926907258,MEDIUM_RISK,MANUAL_REVIEW,0
0.8189314972038578,LOW_RISK,AUTO_APPROVE,0
0.7619489123130659,LOW_RISK,AUTO_APPROVE,1
0.860815429600706,LOW_RISK,AUTO_APPROVE,1
0.7479444308133105,MEDIUM_RISK,MANUAL_REVIEW,1
0.7740060396057498,LOW_RISK,AUTO_APPROVE,1


### Gold Phase 2 - Production Risk Scoring
Purpose: Apply the trained model to all loan applications and generate final approval decisions.


In [0]:
# Apply trained model to full Silver dataset for production scoring

df_silver_full = spark.table("silver_loan_features")

df_silver_full_clean = df_silver_full.dropna(subset=feature_cols)

df_ml_full = assembler.transform(df_silver_full_clean)

full_predictions = lr_model.transform(df_ml_full)



In [0]:
# Apply trained model to full Silver dataset for production scoring


print("Rows before:", df_silver_full.count())
print("Rows after:", df_silver_full_clean.count())


Rows before: 543
Rows after: 529


In [0]:
#Extract Probabilities for ALL Applicants

from pyspark.ml.functions import vector_to_array

full_predictions = full_predictions.withColumn(
    "approval_probability",
    vector_to_array(col("probability"))[1]
)


In [0]:
#Risk Bands for FULL DATA

full_predictions = full_predictions.withColumn(
    "risk_band",
    when(col("approval_probability") >= 0.75, "LOW_RISK")
    .when(col("approval_probability") >= 0.50, "MEDIUM_RISK")
    .otherwise("HIGH_RISK")
)


In [0]:
#Final Loan Decisions (Production)

full_predictions = full_predictions.withColumn(
    "decision",
    when(col("risk_band") == "LOW_RISK", "AUTO_APPROVE")
    .when(col("risk_band") == "MEDIUM_RISK", "MANUAL_REVIEW")
    .otherwise("REJECT")
)


In [0]:
# Store final production-level decisions in Gold Delta table

df_gold_final = full_predictions.select(
    "Loan_ID",
    "approval_probability",
    "risk_band",
    "decision"
)

df_gold_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_loan_decisions")

display(spark.sql("SELECT * FROM gold_loan_decisions"))



Loan_ID,approval_probability,risk_band,decision
LP001003,0.8156742565682977,LOW_RISK,AUTO_APPROVE
LP001005,0.7834368448439682,LOW_RISK,AUTO_APPROVE
LP001006,0.8045651696015323,LOW_RISK,AUTO_APPROVE
LP001008,0.8065374964118932,LOW_RISK,AUTO_APPROVE
LP001011,0.7522611818572699,LOW_RISK,AUTO_APPROVE
LP001013,0.8039618349720238,LOW_RISK,AUTO_APPROVE
LP001014,0.08273955639385733,HIGH_RISK,REJECT
LP001018,0.7790148943898874,LOW_RISK,AUTO_APPROVE
LP001020,0.7915693619398406,LOW_RISK,AUTO_APPROVE
LP001024,0.8308028605873387,LOW_RISK,AUTO_APPROVE


### Gold Phase 3 - Policy Simulation & Business Analytics
Purpose: Evaluate alternative approval strategies and analyze their business impact using SQL analytics.


In [0]:
# Simulate alternative approval policies to analyze business impact (Strict vs Lenient)

df_base = spark.table("gold_loan_decisions")

df_strict = df_base.withColumn(
    "decision_strict",
    when(col("approval_probability") >= 0.85, "AUTO_APPROVE")
    .when(col("approval_probability") >= 0.60, "MANUAL_REVIEW")
    .otherwise("REJECT")
)

df_lenient = df_base.withColumn(
    "decision_lenient",
    when(col("approval_probability") >= 0.65, "AUTO_APPROVE")
    .when(col("approval_probability") >= 0.45, "MANUAL_REVIEW")
    .otherwise("REJECT")
)

In [0]:
#Policy Impact Summary

policy_summary = (
    df_base
    .join(df_strict.select("Loan_ID", "decision_strict"), "Loan_ID")
    .join(df_lenient.select("Loan_ID", "decision_lenient"), "Loan_ID")
)

display(policy_summary)


Loan_ID,approval_probability,risk_band,decision,decision_strict,decision_lenient
LP001003,0.8156742565682977,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001005,0.7834368448439682,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001006,0.8045651696015323,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001008,0.8065374964118932,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001011,0.7522611818572699,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001013,0.8039618349720238,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001014,0.08273955639385733,HIGH_RISK,REJECT,REJECT,REJECT
LP001018,0.7790148943898874,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001020,0.7915693619398406,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001024,0.8308028605873387,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE


In [0]:
# Compare approval outcomes across baseline, strict, and lenient policies

display(df_base.groupBy("decision").count())
display(df_strict.groupBy("decision_strict").count())
display(df_lenient.groupBy("decision_lenient").count())


decision,count
AUTO_APPROVE,417
REJECT,80
MANUAL_REVIEW,32


decision_strict,count
MANUAL_REVIEW,423
REJECT,82
AUTO_APPROVE,24


decision_lenient,count
AUTO_APPROVE,445
REJECT,79
MANUAL_REVIEW,5


In [0]:
# Persist policy simulation results into Gold table for SQL analytics

df_gold_with_policies = (
    df_base
    .join(df_strict.select("Loan_ID", "decision_strict"), on="Loan_ID")
    .join(df_lenient.select("Loan_ID", "decision_lenient"), on="Loan_ID")
)

df_gold_with_policies.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold_loan_decisions")

display(spark.sql("SELECT * FROM gold_loan_decisions"))


Loan_ID,approval_probability,risk_band,decision,decision_strict,decision_lenient
LP001003,0.8156742565682977,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001005,0.7834368448439682,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001006,0.8045651696015323,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001008,0.8065374964118932,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001011,0.7522611818572699,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001013,0.8039618349720238,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001014,0.08273955639385733,HIGH_RISK,REJECT,REJECT,REJECT
LP001018,0.7790148943898874,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001020,0.7915693619398406,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE
LP001024,0.8308028605873387,LOW_RISK,AUTO_APPROVE,MANUAL_REVIEW,AUTO_APPROVE


## SQL Analytics – Gold Layer Insights
Purpose: Analyze loan approval decisions using SQL for business reporting


In [0]:
%sql
--What % of applicants are auto-approved?
-- Approval rate percentage

SELECT
  ROUND(
    SUM(CASE WHEN decision = 'AUTO_APPROVE' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
    2
  ) AS approval_rate_percent
FROM gold_loan_decisions;


approval_rate_percent
78.83


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
--How decisions vary across risk bands?

SELECT
  risk_band,
  decision,
  COUNT(*) AS cases
FROM gold_loan_decisions
GROUP BY risk_band, decision
ORDER BY risk_band, cases DESC;


risk_band,decision,cases
HIGH_RISK,REJECT,80
LOW_RISK,AUTO_APPROVE,417
MEDIUM_RISK,MANUAL_REVIEW,32


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- How many decisions change under stricter rules?

SELECT
  COUNT(*) AS changed_decisions
FROM gold_loan_decisions
WHERE decision <> decision_strict;


changed_decisions
395


In [0]:
%sql
-- Breakdown of changes

SELECT
  decision AS baseline_decision,
  decision_strict,
  COUNT(*) AS cases
FROM gold_loan_decisions
WHERE decision <> decision_strict
GROUP BY decision, decision_strict;


baseline_decision,decision_strict,cases
AUTO_APPROVE,MANUAL_REVIEW,393
MANUAL_REVIEW,REJECT,2


####Delta Lake Optimization (Gold Layer)
Purpose: Improve query performance for analytical workloads


In [0]:
%sql
-- Optimize Gold table for efficient analytics
OPTIMIZE gold_loan_decisions
ZORDER BY (risk_band, decision);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 7449), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1769802945124, 1769802947228, 8, 0, null, List(0, 0), null, 6, 6, 0, 0, null)"


In [0]:
%sql
-- Compute statistics for query planner
ANALYZE TABLE gold_loan_decisions COMPUTE STATISTICS;


## Gold Layer Completed

✔ ML model trained and evaluated  
✔ Production-level risk scoring completed  
✔ Policy simulations executed  
✔ SQL analytics generated  
✔ Delta Lake optimization applied  

The Gold layer now provides a decision-ready, analytics-optimized dataset for business use.
